**This notebook has been written to provide an overview of the kapow module and compare its usage to the standard openmm python library.**
We start by importing the required modules.

In [ ]:
import kapow
from simtk import openmm

The openmm module is the standard python library shipped with the OpenMM simulation suite. The kapow module is provided by this package. Superficially the two should look more or less the same. For every class in openmm there is an equivalent class in kapow. **The kapow module aims to be an alternative interface to all of OpenMM's functionality.**  Lets take a closer look at how the modules interact with one another by creating a class instance. In openmm:

In [ ]:
onbf = openmm.NonbondedForce()

we can then wrap this object with a kapow object:

In [ ]:
knbf = kapow.NonbondedForce.Wrap(onbf)

The call to kapow.NonbondedForce.Wrap creates an instance that provides a different interface to the openmm instance. Any changes made to knbf will be reflect in knbf and vice versa. From the kapow instance you can access the original openmm instance as the wrapped_object attribute:

In [ ]:
assert knbf.wrapped_object is onbf

Instead of doing this in two separate steps you can also call kapow.NonbondedForce() which will create a new openmm instance to wrap automatically. I've prefixed each of the instances with k and o for kapow and openmm respectively to distinguish between the two. To expose the differences lets look at how the attributes of these two differ. Ignoring magic methods:

In [ ]:
print('Module - OpenMM')
for attr in dir(onbf):
    if not attr.startswith('_'):
        print(attr)

print('\nModule - Kapow')
for attr in dir(knbf):
    if not attr.startswith('_'):
        print(attr)

Notice the pattern? The kapow class has far fewer attributes, it also doesn't have any attributes that begin with get or set. Instead for every quantity controlled by a get and set attribute in openmm there is a single attribute in kapow. As a simple example, for the potential cutoff distance the openmm instance uses:

In [ ]:
print('cutoff', onbf.getCutoffDistance())
onbf.setCutoffDistance(0.8)
print('cutoff', onbf.getCutoffDistance())

Whilst the kapow instance uses:

In [ ]:
print('cutoff', knbf.cutoffDistance)
knbf.cutoffDistance = 1.0
print('cutoff', knbf.cutoffDistance)

The difference in the code here again is fairly small, at least for this simple example. Usage of the kapow interface is slightly more clean and compact, but more important is the principle that using get and set methods is considered poor practice in Python (see [here](https://eli.thegreenplace.net/2009/02/06/getters-and-setters-in-python) for a discussion on why). Kapow has taken the methods 'getCutoffDistance' and 'setCutoffDistance' and replaced them with a single property. This is a straightforward case where the get and set methods deal with a single value. Commonly however get and set methods are used to manage access to array elements requiring more sophisticated treatment.

In addition to get and set methods, to manage arrays we also need to be able to determine the number of elements of the array and, optionally, have routines to add and remove elements. As an example lets look at particles for the NonbondedForce. In the openmm NonbondedForce we have the below methods:

In [ ]:
onbf.getNumParticles
onbf.addParticle
onbf.getParticleParameters
onbf.setParticleParameters

This is replaced by a single attribute in the kapow class:

In [ ]:
print(knbf.particles)
type(knbf.particles)

knbf.particles is a custom object provided by Kapow that can be thought of as an extension to the Python property object. In short, it provides an interface which looks and behaves a lot like a Python list but which under the hood is calling the above methods of onbf. A practical example is probably best. Before doing anything interesting we need to add some particles to the NonbondedForce. With the openmm interface:

In [ ]:
onbf.addParticle(1., 1., 1.)

and the equivalent call for kapow:

In [ ]:
knbf.particles.append((1., 1., 1.))

As onbf and knbf are different interfaces for the same object, we should now have two particles. We can check this with:

In [ ]:
print(onbf.getNumParticles())
# vs
print(len(knbf.particles))


Because knbf.particles attempts to behave like a list we can ask for its length. In this case, the \_\_len\_\_ method of knbf.particles is simply wrapping a call to onbf.getNumparticles(). Intuitively, we can also index knbf.particles:

In [ ]:
p = knbf.particles[0]
print(p)
print(type(p))

Now we see something interesting! Compare this with the openmm object.

In [ ]:
print(onbf.getParticleParameters(0))

Rather than returning a simple list, indexing knbf.particles returns a custom Particle object. The Particle object was constructed automatically based on the parameters of the addParticle method. This helps us by providing a much more informative return value and some other handy features as we'll see shortly. The Particle object is derived from a [namedtuple](https://docs.python.org/2/library/collections.html#collections.namedtuple) so you can always use a tuple (or any iterable) in its place meaning this approach also maintains a good level of flexibility.

Now my favorite feature. You can iterate a list, so you can iterate knbf.particles:

In [ ]:
for p in knbf.particles:
    print(p)

The equivalent code for this with the original interface is quite hideous from a pythonic point of view:

In [ ]:
for i in range(onbf.getNumParticles()):
    print(onbf.getParticleParameters(i))

Finally, an example combining iteration with the ability to set elements:

In [ ]:
for i, p in enumerate(knbf.particles):
    knbf.particles[i] = p._replace(sigma=2.)

Here, \_replace is a handy method of namedtuples. The equivalent openmm code is much less compact and readable:

In [ ]:
for i in range(onbf.getNumParticles()):
    charge, sigma, epsilon = onbf.getParticleParameters(i)
    onbf.setParticleParameters(i, charge, 2., epsilon)

ArrayWrappers such as knbf.particles also have detailed help strings that describe the methods that they wrap

In [ ]:
help(knbf.particles)

In addition to ArrayWrappers, another core piece of functionality from the kapow module is the DictWrapper. ArrayWrappers allow a neat way to simpify underlying C++ structures that support indexing. DictWrappers expand this to include key-value pair data structures. This is particularly useful for classes using names for attributes. Consider the below code to examine the default global variables for an AMDIntegrator with openmm:

In [ ]:
amd = openmm.AMDIntegrator(0., 0, 0)
for i in range(amd.getNumGlobalVariables()):
    print(amd.getGlobalVariableName(i), amd.getGlobalVariable(i))

Each global variable has a name but getting hold of that name and the value that goes along with it is rather laborious. In the kapow module these are instead treated as key-value pairs in a dictionary-like interface:

In [ ]:
amd = kapow.AMDIntegrator(0., 0., 0)
print(amd.globalVariables.items())